In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt
import geojson
import requests
import json


In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.ny_map
df = pd.DataFrame(list(db.housing_summary.find()))
df

,_id,month,region_name,inventory,median_days_on_market,median_sale_price,price_drops
0,6096389df954589d78aead94,01,"Albany County, NY",661.750000,58.658333,218320.825000,30.083333
1,6096389df954589d78aead95,01,"Albany, NY metro area",2511.833333,61.941667,206972.016667,97.250000
2,6096389df954589d78aead96,01,"Allegany County, NY",127.750000,95.091667,65026.925000,2.833333
3,6096389df954589d78aead97,01,"Amsterdam, NY metro area",156.750000,85.541667,106653.366667,5.250000
4,6096389df954589d78aead98,01,"Auburn, NY metro area",212.750000,74.375000,115040.266667,7.250000
...,...,...,...,...,...,...,...
619,6096389df954589d78aeafff,12,"Warren County, NY",244.083333,61.991667,232587.141667,5.083333
620,6096389df954589d78aeb000,12,"Washington County, NY",190.416667,81.308333,167634.916667,4.083333
621,6096389df954589d78aeb001,12,"Wayne County, NY",82.000000,17.741667,155767.033333,2.583333
622,6096389df954589d78aeb002,12,"Westchester County, NY",2545.083333,41.150000,601293.875000,55.000000


In [3]:
df['region_name']= df['region_name'].str.split(',',expand=True)
df = df[df['region_name'].str.contains('(?!$)County')]
df= df.groupby(['region_name']).agg({'inventory':'mean',
                         'median_days_on_market':'mean',
                         'median_sale_price':'mean',
                         'price_drops':'mean',
                        })
df

,inventory,median_days_on_market,median_sale_price,price_drops
region_name,,,,
Albany County,609.545833,27.030972,2.300227e+05,28.687500
Allegany County,102.175000,91.994167,8.706322e+04,2.651389
Bronx County,1193.536111,77.600139,5.177711e+05,29.979167
Cattaraugus County,239.420833,74.701806,1.257773e+05,7.968056
Cayuga County,200.584722,69.873056,1.385143e+05,6.911111
Chautauqua County,318.287500,64.660417,1.070669e+05,10.513889
Dutchess County,598.369444,63.075833,3.303626e+05,17.887500
Erie County,1092.451389,17.060694,1.790064e+05,53.261111
Fulton County,193.761111,89.581806,1.199528e+05,8.305556


In [4]:
response = requests.get("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/new-york-counties.geojson").json()
print(json.dumps(response ,indent = 4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": [
                    [
                        [
                            [
                                -74.82010799999999,
                                43.818507
                            ],
                            [
                                -74.854171,
                                44.070088999999996
                            ],
                            [
                                -74.28187,
                                44.120551999999996
                            ],
                            [
                                -74.25599799999999,
                                43.969797
                            ],
                            [
                                -74.336826,
                                43.925222999999995
                            ],
             

In [5]:
response["features"][0]['properties']

{'geoid': '05000US36041', 'name': 'Hamilton County'}

In [6]:
 for row in df.iterrows():
        print(row[1][0])

609.5458333333333
102.175
1193.536111111111
239.42083333333335
200.5847222222222
318.28749999999997
598.3694444444444
1092.451388888889
193.7611111111111
64.18888888888888
5274.552777777778
113.60972222222222
166.81805555555556
799.7458333333334
158.6611111111111
6464.018055555555
7679.501388888889
354.55
824.4041666666666
244.73194444444445
1388.8208333333332
69.51666666666667
263.79861111111114
433.12500000000006
6006.251388888889
365.69166666666666
1647.6805555555557
942.2805555555556
783.4319444444445
403.97499999999997
100.64166666666667
4335.440277777778
528.1388888888889
323.2138888888889
264.1486111111111
133.5513888888889
3077.966666666667
69.65972222222223


In [7]:
redfin_regions = [region for region in df.index]
redfin_regions

['Albany County',
 'Allegany County',
 'Bronx County',
 'Cattaraugus County',
 'Cayuga County',
 'Chautauqua County',
 'Dutchess County',
 'Erie County',
 'Fulton County',
 'Genesee County',
 'Kings County',
 'Livingston County',
 'Madison County',
 'Monroe County',
 'Montgomery County',
 'Nassau County',
 'New York County',
 'Niagara County',
 'Onondaga County',
 'Ontario County',
 'Orange County',
 'Orleans County',
 'Oswego County',
 'Putnam County',
 'Queens County',
 'Rensselaer County',
 'Richmond County',
 'Rockland County',
 'Saratoga County',
 'Schenectady County',
 'Schoharie County',
 'Suffolk County',
 'Sullivan County',
 'Warren County',
 'Washington County',
 'Wayne County',
 'Westchester County',
 'Wyoming County']

In [8]:
for i, row in df.iterrows():
    print(row)
#     if NAME == row[1][0]:
#         new_geo['features'][i]['properties']["median_sale_price"] = round(row[1][3],2)
#         new_geo['features'][i]['properties']["price_drops"] = round(row[1][4],2)
#         new_geo['features'][i]['properties']["inventory"] = round(row[1][1],2)
#         new_geo['features'][i]['properties']["median_days_on_market"] = round(row[1][2],2)

inventory                   609.545833
median_days_on_market        27.030972
median_sale_price        230022.653611
price_drops                  28.687500
Name: Albany County, dtype: float64
inventory                  102.175000
median_days_on_market       91.994167
median_sale_price        87063.219167
price_drops                  2.651389
Name: Allegany County, dtype: float64
inventory                  1193.536111
median_days_on_market        77.600139
median_sale_price        517771.120556
price_drops                  29.979167
Name: Bronx County, dtype: float64
inventory                   239.420833
median_days_on_market        74.701806
median_sale_price        125777.314583
price_drops                   7.968056
Name: Cattaraugus County, dtype: float64
inventory                   200.584722
median_days_on_market        69.873056
median_sale_price        138514.338750
price_drops                   6.911111
Name: Cayuga County, dtype: float64
inventory                   318.287500

In [9]:
new_geo = response

redfin_regions = [region for region in df.index]

for i in range(len(response['features'])):
    NAME = response["features"][i]['properties']['name']
    if NAME in redfin_regions:
#         print("region found!")
        for index, row in df.iterrows():
            if NAME == index:
                new_geo['features'][i]['properties']["inventory"] = round(row[0],2)
                new_geo['features'][i]['properties']["median_days_on_market"] = round(row[1],2)
                new_geo['features'][i]['properties']["median_sale_price"] = round(row[2],2)
                new_geo['features'][i]['properties']["price_drops"] = round(row[3],2)
    else:
#         print("region not found!")
        new_geo['features'][i]['properties']["median_sale_price"] = 0
        new_geo['features'][i]['properties']["price_drops"] = 0
        new_geo['features'][i]['properties']["inventory"] = 0
        new_geo['features'][i]['properties']["median_days_on_market"] = 0

for feature in new_geo["features"]:
    print(feature["properties"])

{'geoid': '05000US36041', 'name': 'Hamilton County', 'median_sale_price': 0, 'price_drops': 0, 'inventory': 0, 'median_days_on_market': 0}
{'geoid': '05000US36017', 'name': 'Chenango County', 'median_sale_price': 0, 'price_drops': 0, 'inventory': 0, 'median_days_on_market': 0}
{'geoid': '05000US36051', 'name': 'Livingston County', 'inventory': 113.61, 'median_days_on_market': 40.57, 'median_sale_price': 149107.69, 'price_drops': 4.47}
{'geoid': '05000US36097', 'name': 'Schuyler County', 'median_sale_price': 0, 'price_drops': 0, 'inventory': 0, 'median_days_on_market': 0}
{'geoid': '05000US36095', 'name': 'Schoharie County', 'inventory': 100.64, 'median_days_on_market': 83.49, 'median_sale_price': 154512.64, 'price_drops': 2.99}
{'geoid': '05000US36083', 'name': 'Rensselaer County', 'inventory': 365.69, 'median_days_on_market': 38.53, 'median_sale_price': 200804.76, 'price_drops': 16.33}
{'geoid': '05000US36065', 'name': 'Oneida County', 'median_sale_price': 0, 'price_drops': 0, 'invent

In [10]:
#load in totals
db.ny_choropleth.drop()
db.ny_choropleth.find()
db.ny_choropleth.insert_one(new_geo)
